<a href="https://colab.research.google.com/github/HoseinNekouei/Data_Augmentation_GPT/blob/main/GPT_Data_Augmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install openai
! pip install datasets transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.5/328.5 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 21.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.9 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninsta

In [ ]:
from google.colab import userdata
import openai

# Add GPT API key in the secrect manager
OPENAI_API_key= userdata.get('OPENAI_API_KEY')
openai.api_key= OPENAI_API_key
check_point='gpt-3.5-turbo-instruct'

In [ ]:
from datasets import load_dataset
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:


def load_data(dataset_url):
    """
    Loads and processes a dataset of paraphrased tweets from a CSV file.

    This function loads the dataset from the provided URL, filters it to include
    only relevant columns ('text' and 'airline_sentiment'), and returns the processed dataset.

    Args:
        dataset_url (str): The URL or file path to the CSV file containing the dataset.

    Returns:
        DatasetDict: A dictionary-like object containing the loaded and processed dataset.

    Example:
        dataset_url = '/content/drive/MyDrive/dataset/US_airline/Parapherized_Tweets.csv'
        raw_data = load_paraphrized_tweets(dataset_url)
        print(raw_data['train'][:3])  # Print the first 3 records in the training set
    """

    # Load the CSV file in memory
    raw_data = load_dataset('csv', data_files=dataset_url, sep=',')

    # raw_data['train']= raw_data['train'].filter(lambda x: x['airline_sentiment_confidence']>= 0.65 )
    raw_data['train']= raw_data['train'].filter(lambda x: len(x['text']) > 30)

    # Select only the 'text' and 'airline_sentiment' columns
    raw_data['train']= raw_data['train'].select_columns(['text', 'airline_sentiment', 'model'])

    return raw_data

raw_dataset_url= '/content/drive/MyDrive/Projects/Sentiment_Analysis/gpt_evaluated_parapherized.csv'
raw_data= load_data(raw_dataset_url)
raw_data

Generating train split: 0 examples [00:00, ? examples/s]

Filter:   0%|          | 0/26279 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'airline_sentiment', 'model'],
        num_rows: 26276
    })
})

In [ ]:
def paraphraze_text(text, sentiment, max_token= 34):
  prompt= f'paraphrase the following text based on {sentiment} feeling: \n\n {text}\n\n paraphrazed version:'
  response= openai.completions.create(
      model= check_point,
      prompt= prompt,
      max_tokens= max_token,
      n=1,
      stop=None,
      temperature= 0.7
  )

  return response

In [ ]:
# openai.models.list()

In [ ]:
def get_tweets(raw_data, sentiment):
    """
    Filters and returns tweets with sentiment from the given dataset.

    Args:
        raw_data (DataFrame): The dataset containing tweets. It is expected to have a 'train'
                              column that includes a list of tweet dictionaries, and each
                              tweet dictionary should have an 'airline_sentiment' key.

    Returns:
        list: A list of dictionaries, each representing a tweet with a neutral sentiment.

    Example:
        raw_data = {
            'train': [
                {'airline_sentiment': 'neutral', 'text': 'Flight was okay.'},
                {'airline_sentiment': 'positive', 'text': 'Great service!'},
                {'airline_sentiment': 'neutral', 'text': 'No issues, decent experience.'},
            ]
        }

        neutral_tweets = get_neutral_tweets(raw_data)
        for tweet in neutral_tweets[:3]:
            print(tweet)
    """

    parapherized_tweets= [item for item in raw_data['train'] if item['airline_sentiment']== sentiment and item['model']=='gpt_evaluated']

    return parapherized_tweets

In [ ]:
from IPython.display import clear_output

def main(orginal_texts, debug=False):

  for index, tweet in enumerate(orginal_texts[:4500]):

    response= paraphraze_text(tweet['text'], tweet['airline_sentiment'])

    # Create a dictionary to store the paraphrased text and its sentiment
    paraphrazed_tweet = {'text': response.choices[0].text.strip(), 'airline_sentiment': tweet['airline_sentiment'], 'model':' GPT-3.5-Turbo'}

    # Add the paraphrased text and sentiment to the training dataset
    raw_data['train'] = raw_data['train'].add_item(paraphrazed_tweet)

    # Print a message indicating that the paraphrased text has been added to the dataset
    print(f"[Tweet {(index + 1)}]: {paraphrazed_tweet}")


    if (index + 1) % 50 == 0:

      # Clear the output to display the updated message
      clear_output()

      # Save the updated dataset to a CSV file
      raw_data['train'].to_csv('/content/drive/MyDrive/Projects/Sentiment_Analysis/gpt_evaluated_parapherized.csv')

      print(f'Dataset Updated! on index: {index+1}')


In [ ]:
import matplotlib.pyplot as plt

def show_result():

  orginal_dataset_url= '/content/drive/MyDrive/Projects/Sentiment_Analysis/gpt_evaluated_update.csv'
  orginal_dataset = load_dataset('csv', data_files= orginal_dataset_url)

  augmented_dataset_url= '/content/drive/MyDrive/Projects/Sentiment_Analysis/gpt_evaluated_parapherized.csv'
  augmented_dataset = load_dataset('csv', data_files= augmented_dataset_url)

  # train_labels = dataset.set_format('pandas')
  df_augmented_dataset= augmented_dataset['train'].to_pandas()

  df_original_dataset= orginal_dataset['train'].to_pandas()

  df_original = df_original_dataset['airline_sentiment'].value_counts()
  df_augmented = df_augmented_dataset['airline_sentiment'].value_counts()

  print(df_original)
  print(df_augmented)

  # Plot the bar chart for train and test label distributions
  plt.figure(figsize=(10, 4))

  plt.subplot(1,2,1)
  plt.bar(df_original.index, df_original.values, color= 'green')
  plt.title('Original dataset')

  plt.subplot(1, 2, 2)
  plt.bar(df_augmented.index, df_augmented.values, color='blue')
  plt.title('Augmented dataset')

  plt.tight_layout()
  plt.show()

In [ ]:
if __name__== '__main__':

  # Example Paraphrase
  orginal_texts= get_tweets(raw_data, 'negative')
  main(orginal_texts)
  show_result()

Creating CSV from Arrow format:   0%|          | 0/31 [00:00<?, ?ba/s]

Dataset Updated! on index: 4450
[Tweet 4451]: {'text': 'Unfortunately, @USAirways has caused a lot of frustration by delaying flight 1881 in Philly since 1:30pm and keeping passengers sitting on the tarmac', 'airline_sentiment': 'negative', 'model': ' GPT-3.5-Turbo'}
[Tweet 4452]: {'text': 'This has been the most terrible trip with @USAirways! I really wish the pilot had allowed me to get off the plane at 12pm during the first delay', 'airline_sentiment': 'negative', 'model': ' GPT-3.5-Turbo'}
[Tweet 4453]: {'text': "It's incredibly frustrating that @USAirways requires speaking to an agent in order to redeem a companion fare. The price difference of $99 versus $1200 is outrageous", 'airline_sentiment': 'negative', 'model': ' GPT-3.5-Turbo'}
[Tweet 4454]: {'text': 'Can I please receive a response, @USAirways? I attempted to speak with the gate attendant (who directed me to the incorrect location for customer service) and finally', 'airline_sentiment': 'negative', 'model': ' GPT-3.5-Turb